In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    TopKLogitsWarper,
    TemperatureLogitsWarper,
    StoppingCriteriaList,
    MaxLengthCriteria,
    XGLMForCausalLM,
)
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")

In [2]:
model.cuda()

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_pr

In [23]:
def get_response(prompt_text, use_gpu=False):
    input_prompt = f'คำสุภาพของ \"{prompt_text}\" คือ'
    input_ids = tokenizer(input_prompt, return_tensors="pt", truncation=True, max_length=256).input_ids
    if use_gpu:
        input_ids = input_ids.cuda()

    # instantiate logits processors
    logits_processor = LogitsProcessorList(
        [
            MinLengthLogitsProcessor(15, eos_token_id=model.generation_config.eos_token_id),
        ]
    )
    # # instantiate logits processors
    # logits_warper = LogitsProcessorList(
    #     [
    #         TopKLogitsWarper(50),
    #         TemperatureLogitsWarper(0.7),
    #     ]
    # )

    # stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=150)])

    torch.manual_seed(0)
    # outputs = model.sample(
    #     input_ids,
    #     logits_processor=logits_processor,
    #     logits_warper=logits_warper,
    #     stopping_criteria=stopping_criteria,
    # )
    
    output = model.generate(input_ids=input_ids, top_k=50, temperature=0.7, max_length=256, early_stopping=True, no_repeat_ngram_size=2,
                            # min_length=len(input_ids),
                            # num_beams=5, no_repeat_ngram_size=2, early_stopping=True, max_new_tokens=100,
                            logits_processor=logits_processor)


    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    torch.cuda.empty_cache()
    return generated_text


In [4]:
# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(
            15, eos_token_id=model.generation_config.eos_token_id),
    ]
)
# instantiate logits processors
logits_warper = LogitsProcessorList(
    [
        TopKLogitsWarper(50),
        TemperatureLogitsWarper(0.7),
    ]
)

stopping_criteria = StoppingCriteriaList(
    [MaxLengthCriteria(max_length=150)])


In [18]:
tokens = tokenizer("คำสุภาพของ \"กูกลับ พน อีเดาะ\" คือ",
                   return_tensors="pt", truncation=True, max_length=256).input_ids
output = model.generate(input_ids=tokens.cuda(), top_k=50, temperature=0.7, max_length=256, early_stopping=True, no_repeat_ngram_size=2,
                        # min_length=len(input_ids),
                        # num_beams=5, no_repeat_ngram_size=2, early_stopping=True, max_new_tokens=100,
                        logits_processor=logits_processor)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


คําสุภาพของ "กูกลับ พน อีเดาะ" คือ "ไม่เอาพวกมึง"


In [19]:
import pandas as pd

test_df = pd.read_csv("data/test.csv")
test_df = test_df[test_df["label"] == 0]

In [24]:
from tqdm import tqdm
polite = []
for _, row in tqdm(test_df.iterrows()):
    polite.append(get_response(row["text"], use_gpu=True))

190it [01:20,  1.45it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
791it [04:52,  4.67it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
960it [05:46,  4.62it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
986it [06:00,  1.11it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
1333it [08:22,  5.40it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
1362it [08:30,  2.95it/s]Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpe

In [25]:
polite_filtered = []
for p in polite:
    idx = p.find("คือ")
    text = p[idx+3:].strip()
    polite_filtered.append(text)

In [26]:
sentence_pair = pd.DataFrame(
    {'impolite': test_df.text,
     'polite': polite_filtered}
)

In [27]:
sentence_pair.to_csv("result.csv", index=False)